In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [11]:
df = pd.read_csv("netflix_new/netflix_titles.csv")


In [ ]:
df = pd.read_csv("netflix_new/netflix_titles.csv")

In [12]:
print(df.head(5))

  show_id     type                  title         director  \
0      s1    Movie   Dick Johnson Is Dead  Kirsten Johnson   
1      s2  TV Show          Blood & Water              NaN   
2      s3  TV Show              Ganglands  Julien Leclercq   
3      s4  TV Show  Jailbirds New Orleans              NaN   
4      s5  TV Show           Kota Factory              NaN   

                                                cast        country  \
0                                                NaN  United States   
1  Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...   South Africa   
2  Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...            NaN   
3                                                NaN            NaN   
4  Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...          India   

           date_added  release_year rating   duration  \
0  September 25, 2021          2020  PG-13     90 min   
1  September 24, 2021          2021  TV-MA  2 Seasons   
2  September 24, 2021        

In [13]:
pd.set_option('display.max_colwidth', None)

In [15]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB
None


** Count null values in each column

In [17]:
print(df.isnull().sum())

show_id            0
type               0
title              0
director        2634
cast             825
country          831
date_added        10
release_year       0
rating             4
duration           3
listed_in          0
description        0
dtype: int64


Handling Director, Cast, and Country Column These columns have a large number of missing values. Dropping thousands of rows would destroy our dataset.Since these are categorical fields, we can fill the missing values with a placeholder string like "Unknown". This preserves the rows for other analyses and allows us to filter for or ignore "Unknown" values later if needed.

In [22]:
df['director'].fillna('Unknown')
df['cast'].fillna('Unknown')
df['country'].fillna('Unknown')

0       United States
1        South Africa
2             Unknown
3             Unknown
4               India
            ...      
8802    United States
8803          Unknown
8804    United States
8805    United States
8806            India
Name: country, Length: 8790, dtype: object

Handling date_added, rating, and duration.These columns have very few missing values (typically < 15). Because these values are essential for specific analyses (time-series, audience targeting, duration analysis) and the number of missing rows is tiny compared to the total dataset size (~0.1%), the simplest and cleanest approach is to drop these rows. The impact on the overall dataset is negligible.

In [23]:
df.dropna(subset=['date_added', 'rating', 'duration'], inplace=True)

In [24]:
print(df.isnull().sum())

show_id         0
type            0
title           0
director        0
cast            0
country         0
date_added      0
release_year    0
rating          0
duration        0
listed_in       0
description     0
dtype: int64


The date_added column is a string, so you can't perform time-based calculations on it.so we convert this into proper datetime object

In [25]:
df['date_added'] = pd.to_datetime(df['date_added'].str.strip())

Similarly The duration column is messy. It contains " min" for movies and " Season(s)" for TV shows, all in the same column as strings. It's best to split this into two new numerical columns: one for movie duration in minutes and one for the number of seasons for a TV show. This makes analysis much cleaner.


In [26]:
# Initialize new columns with NaN
df['movie_duration_mins'] = np.nan
df['tv_show_seasons'] = np.nan

# For Movies, extract the number and fill the movie_duration_mins column
is_movie = df['type'] == 'Movie'
df.loc[is_movie, 'movie_duration_mins'] = df.loc[is_movie, 'duration'].str.replace(' min', '').astype(int)

# For TV Shows, extract the number and fill the tv_show_seasons column
is_tv_show = df['type'] == 'TV Show'
# Use .replace for both ' Season' and ' Seasons'
df.loc[is_tv_show, 'tv_show_seasons'] = df.loc[is_tv_show, 'duration'].str.replace(' Seasons', '').str.replace(' Season', '').astype(int)

Extract more information from existing columns to aid EDA.

Extract Year and Month from date_added

In [27]:
df['year_added'] = df['date_added'].dt.year
df['month_added'] = df['date_added'].dt.month

In [28]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 8790 entries, 0 to 8806
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   show_id              8790 non-null   object        
 1   type                 8790 non-null   object        
 2   title                8790 non-null   object        
 3   director             8790 non-null   object        
 4   cast                 8790 non-null   object        
 5   country              8790 non-null   object        
 6   date_added           8790 non-null   datetime64[ns]
 7   release_year         8790 non-null   int64         
 8   rating               8790 non-null   object        
 9   duration             8790 non-null   object        
 10  listed_in            8790 non-null   object        
 11  description          8790 non-null   object        
 12  movie_duration_mins  6126 non-null   float64       
 13  tv_show_seasons      2664 non-null   f

In [29]:
df['content_source'] = np.where(df['release_year'] == df['year_added'], 'Original', 'Licensed')

In [30]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 8790 entries, 0 to 8806
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   show_id              8790 non-null   object        
 1   type                 8790 non-null   object        
 2   title                8790 non-null   object        
 3   director             8790 non-null   object        
 4   cast                 8790 non-null   object        
 5   country              8790 non-null   object        
 6   date_added           8790 non-null   datetime64[ns]
 7   release_year         8790 non-null   int64         
 8   rating               8790 non-null   object        
 9   duration             8790 non-null   object        
 10  listed_in            8790 non-null   object        
 11  description          8790 non-null   object        
 12  movie_duration_mins  6126 non-null   float64       
 13  tv_show_seasons      2664 non-null   f

Save the Cleaned Data

In [31]:
df.to_csv('netflix_cleaned.csv', index=False)
print("\nCleaned data saved to 'netflix_cleaned.csv'")


Cleaned data saved to 'netflix_cleaned.csv'
